In [1]:
import yfinance as yf
import pandas as pd

# Define the ticker symbol for Bitcoin
btc_ticker = "BTC-USD"

# Define the date range
start_date = "2024-12-19"
end_date = "2025-2-1"

# Fetch the data using yfinance
btc_data = yf.download(btc_ticker, start=start_date, end=end_date)

# Reset the index to make 'Date' a column
btc_data.reset_index(inplace=True)

# Select and reorder columns for consistency with your request
btc_data = btc_data[["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]]

# Format the 'Date' column to match the required structure
btc_data["Date"] = btc_data["Date"].dt.strftime("%Y-%m-%d")

# Save to a CSV file
output_file = "BTC_Data_2022-20243.csv"
btc_data.to_csv(output_file, index=False)

print(f"Data has been successfully saved to {output_file}.")

c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed

Data has been successfully saved to BTC_Data_2022-20243.csv.


In [ ]:
import pandas as pd
import talib

# بارگذاری دیتاست از فایل CSV
df = pd.read_csv("BTC-USD.csv")

# نمایش 5 ردیف اول برای بررسی ساختار دیتاست
print(df.head())

# محاسبه EMA با دوره ۱۰
df["EMA_10"] = talib.EMA(df["Close"], timeperiod=10)

# محاسبه RSI با دوره ۱۴
df["RSI_14"] = talib.RSI(df["Close"], timeperiod=14)

# محاسبه MACD و سیگنال
df["MACD"], df["MACD_signal"], df["MACD_hist"] = talib.MACD(
    df["Close"], fastperiod=12, slowperiod=26, signalperiod=9
)

# نمایش نتایج جدید با اندیکاتورها
print(
    df[["Date", "Close", "EMA_10", "RSI_14", "MACD", "MACD_signal", "MACD_hist"]].tail()
)

# ذخیره دیتاست به یک فایل جدید CSV
df.to_csv("BTC_USD_indicators.csv", index=False)

         Date        Open        High         Low       Close   Adj Close  \
0  2014-09-17  465.864014  468.174011  452.421997  457.334015  457.334015   
1  2014-09-18  456.859985  456.859985  413.104004  424.440002  424.440002   
2  2014-09-19  424.102997  427.834991  384.532013  394.795990  394.795990   
3  2014-09-20  394.673004  423.295990  389.882996  408.903992  408.903992   
4  2014-09-21  408.084991  412.425995  393.181000  398.821014  398.821014   

     Volume  
0  21056800  
1  34483200  
2  37919700  
3  36863600  
4  26580100  
            Date          Close        EMA_10     RSI_14         MACD  \
3737  2024-12-10   96675.429688  97794.648908  56.319368  3446.748371   
3738  2024-12-11  101173.031250  98408.900243  63.466202  3483.779130   
3739  2024-12-12  100043.000000  98706.009290  60.775607  3382.945703   
3740  2024-12-13  101459.257812  99206.599930  62.898526  3378.370898   
3741  2024-12-14  101372.968750  99600.485170  62.675951  3329.403238   

      MACD_sig

In [4]:
import pandas as pd
import numpy as np

# خواندن دیتاست
df = pd.read_csv('BTC-USD.csv')

# فیلتر کردن داده‌ها از تاریخ ۲۰۱۸ به بعد
df = df[df["Date"] >= "2018-01-01"]

# تنظیم ستون Date به عنوان اندیس
df = df.set_index('Date')

# -------------------------------
# 1. محاسبه میانگین‌های متحرک
# -------------------------------
df['MA_20'] = df['Close'].rolling(window=20).mean()
df['MA_50'] = df['Close'].rolling(window=50).mean()
df['MA_200'] = df['Close'].rolling(window=200).mean()

# -------------------------------
# 2. محاسبه شاخص قدرت نسبی (RSI)
# -------------------------------
period = 14  # دوره محاسبه RSI
delta = df['Close'].diff()

# محاسبه سود و زیان
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)

# محاسبه میانگین سود و زیان
avg_gain = gain.rolling(window=period, min_periods=period).mean()
avg_loss = loss.rolling(window=period, min_periods=period).mean()

# محاسبه RSI
rs = avg_gain / avg_loss
df['RSI'] = 100 - (100 / (1 + rs))

# رفع مقدار NaN با استفاده از انتساب مستقیم (برای رفع FutureWarning)
df['RSI'] = df['RSI'].fillna(0)

# -------------------------------
# 3. محاسبه MACD، سیگنال و هیستوگرام
# -------------------------------
# محاسبه EMA با دوره‌های 12 و 26
ema12 = df['Close'].ewm(span=12, adjust=False).mean()
ema26 = df['Close'].ewm(span=26, adjust=False).mean()

# MACD
df['MACD'] = ema12 - ema26

# خط سیگنال: EMA9 از MACD
df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

# هیستوگرام MACD: تفاوت MACD و خط سیگنال
df['Histogram'] = df['MACD'] - df['Signal']

# نمایش نمونه‌ای از دیتاست با ستون‌های اضافه شده
print(df[['Open', 'High', 'Low', 'Close', 'Volume', 
          'MA_20', 'MA_50', 'MA_200', 'RSI', 'MACD', 'Signal', 'Histogram']].head(20))



df.to_csv('BTC_USD_INDICATORs.csv')


                    Open          High           Low         Close  \
Date                                                                 
2018-01-01  14112.200195  14112.200195  13154.700195  13657.200195   
2018-01-02  13625.000000  15444.599609  13163.599609  14982.099609   
2018-01-03  14978.200195  15572.799805  14844.500000  15201.000000   
2018-01-04  15270.700195  15739.700195  14522.200195  15599.200195   
2018-01-05  15477.200195  17705.199219  15202.799805  17429.500000   
2018-01-06  17462.099609  17712.400391  16764.599609  17527.000000   
2018-01-07  17527.300781  17579.599609  16087.700195  16477.599609   
2018-01-08  16476.199219  16537.900391  14208.200195  15170.099609   
2018-01-09  15123.700195  15497.500000  14424.000000  14595.400391   
2018-01-10  14588.500000  14973.299805  13691.200195  14973.299805   
2018-01-11  14968.200195  15018.799805  13105.900391  13405.799805   
2018-01-12  13453.900391  14229.900391  13158.099609  13980.599609   
2018-01-13  13952.40